In [2]:
import random
import cv2
from matplotlib import pyplot as plt

import albumentations as A

import sys
import os
import numpy as np

print(A.__version__)

1.3.0


In [3]:
# bbox 포함 크롭
aug_type = "_bsr_crop"
transform = A.Compose(
    [A.BBoxSafeRandomCrop(p=1)],
    bbox_params=A.BboxParams(format='yolo', label_fields=['category_ids'])
)

In [14]:
# 밝기 조절
aug_type = "_bright"
transform = A.Compose(
    [A.RandomBrightnessContrast(brightness_limit = (-0.10, 0.10), p=1)],
    bbox_params=A.BboxParams(format='yolo', label_fields=['category_ids']),
)

In [11]:
# 좌우반전(rot_hf)
aug_type = "_rot_hf" 
transform = A.Compose(
    [A.HorizontalFlip(p=1)],
    bbox_params=A.BboxParams(format='yolo', label_fields=['category_ids']),
)

In [9]:
# 상하반전(rot_vf)
aug_type = "_rot_vf" 
transform = A.Compose(
    [A.VerticalFlip(p=1)],
    bbox_params=A.BboxParams(format='yolo', label_fields=['category_ids']),
)

In [5]:
"""파일 경로 설정 및 읽어오기"""
root_dir = "C:/Users/user/bee_dataset/" # 데이터셋 경로
img_dir = root_dir + "set_origin/train/images/" 
label_dir = root_dir + "set_origin/train/labels/"

# 라벨 폴더 읽어오기
label_file_list = os.listdir(label_dir)

# 이미지 폴더 읽어오기
img_file_list = os.listdir(img_dir)

category_id_to_name = {
    0 : 'AB_LI', 
    1 : 'AB_CA', 
    2 : 'AB_AP', 
    3 : 'AB_BI', 
    4 : 'QB_LI', 
    5 : 'QB_CA', 
    6 : 'QB_AP', 
    7 : 'QB_BI'
}

In [6]:
for img in img_file_list:
    image = cv2.imread(img_dir + img)
    img_width = image.shape[1]
    img_height = image.shape[0]
    
    bboxes = []
    
    with open(label_dir + img[:-4] + ".txt", 'r') as f: # 라벨 txt 파일 내용 "2차원 리스트로" 읽어오기
        bbox = []
        category_ids = []
        
        while True:
            line = f.readline()
            
            if line == "":
                break
            elif '\n' in line:
                line.replace('\n', '')
                
            data = line.split(" ")
            category_ids.append(int(data[0]))
            del data[0]
            
            for d in data:
                 bbox.append(float(d))
                
            bboxes.append(bbox) 
            bbox = []
            
    
    # 증강 파이프라인 설정
    
    # 증강
    try:
        transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)
    except:
        print("오류 이미지: ", img)
    
    # 결과 저장 경로 설정
    img_save_path = root_dir + "set_crop/train/images/"
    label_save_path = root_dir + "set_crop/train/labels/"

    # 증강된 이미지 저장
    cv2.imwrite(img_save_path + img[:-4] + aug_type + ".jpg", transformed["image"])

    # 증강된 바운딩 박스 라벨로 저장
#     q = open(label_save_path + img[:-4] + aug_type + ".txt", 'w')
#     q.close()
    with open(label_save_path + img[:-4] + aug_type + ".txt", 'a') as fi:
        for i in range(len(transformed["bboxes"])):
            fi.write(str(category_ids[i]) + " ")
            fi.write(str(transformed["bboxes"][i][0]) + " ")
            fi.write(str(transformed["bboxes"][i][1]) + " ")
            fi.write(str(transformed["bboxes"][i][2]) + " ")
            fi.write(str(transformed["bboxes"][i][3]) + "\n")
            
print(aug_type, "증강 완료")

오류 이미지:  01_1_R_AB_AP_20220811_03_0128.jpg
_bsr_crop 증강 완료
